# 컴포넌트 최적화
- 렌더링 속도를 개선하고, 불필요한 리렌더링을 줄이는 것


<br>

<hr>

<br>

## 01. 컴포넌트 최적화

<br>

### 성능 최적화
- React 애플리케이션의 성능 최적화는 필수

<br>

#### 1. 일반적인 웹 최적화
- 이미지 최적화
- HTML, CSS, JavaScript 자원 압축
- 웹 브라우저 캐싱 전략
- 지연 로딩
- HTTP/2 프로토콜 사용
  
  $\rightarrow$ **웹 브라우저와 네트워크 환경에 중점을 둔 보편적인 전략**

<br>

#### 2. React 특화 최적화
-  **React의 구조와 렌더링 방식에 대한 이해를 바탕**
   -  React는 컴포넌트 단위로 UI를 구성하고, 상태 변화에 따라 컴포넌트를 리렌더링
   
   $\rightarrow$ 컴포넌트 렌더링을 줄이고, 상태 업데이트를 효율적으로 관리하며, 불필요한 연산을 최소화

   - `React.memo` : 불필요한 리렌더링 방지
   - `React.lazy` : 필요한 시점에 컴포넌트를 불러오는 코드 스플리팅
   - `useMemo`, `useCallback` : 복잡한 연산 결과나 함수를 메모이제이션
   - `useTransition`, `useDeferredvalue` : 상태 업데이트를 비동기적으로 지연 처리

<br>

### 불필요한 리렌더링 
- 컴포넌트는 부모-자식 관계를 맺으며 컴포넌트 트리 구조로 연결
  - `App` 컴포넌트가 루트 컴포넌트일 때, `A`컴포넌트를 렌더링하고, `A`는 `B`를 렌더링, `B`는 `C`를 렌더링한다면 컴포넌트 트리는

<img src='img/1101.png' width=100>

<br>

- **상위 컴포넌트가 리렌더링되면, 그에 연결된 모든 하위 컴포넌트도 함께 리렌더링**
  
  $\rightarrow$ 컴포넌트의 상태나 `props`가 전혀 바뀌지 않았음에도 컴포넌트가 리렌더링 
  
  $\rightarrow$ **불필요한 리렌더링**

  $\rightarrow$ 컴포넌트의 수가 많아질수록 렌더링 비용도 커질 수 있으며, 성능 저하의 원인이 될 수도 있음

<br>

### 메모이제이션 (Memoization)
- **이미 계산한 결과를 저장해두고 같은 계산을 반복해야 할 때, 저장된 값을 다시 사용하는 최적화 기법**
  - 재귀 함수 등 같은 계산이 반복되는 알고리즘에서 효과적

<br>

- **피보나치 수열 계산 함수**
  - **$n$이 1이하일 때까지 재귀적으로 자신을 호출해 피보나치 수를 계산**

```ts
function fibonacci(n) {
   if (n <= 1) {
      return n;
   }
   return fibonacci(n - 1) + fibonacci(n - 2);
}

console.log(fibonacci(50));
```

<br>

- **메모이제이션을 적용한 피보나치 함수**
   - `memo`라는 객체에 이미 계산한 값을 저장
   - 같은 입력이 다시 들어오면 새로운 계산 없이 `memo`에서 값을 가져오기 때문에 중복 계산을 피할 수 있어 성능이 크게 향상

<br>

```ts
function fibonacci(n, memo = {}) {
   if (n <= 1) {
      return n;
   }

   if (meno[n]) {
      return memo[n];
   }

   memo[n] = fibonacci(n - 1 , memo) + fibonacci(n - 2, memo);

   return memo[n];
}

console.log(fibonacci(40));
```

<br>

<hr>

<br>

## 02. 컴포넌트 메모이제이션
- 컴포넌트 자체를 메모이제이션하면, 해당 컴포넌트의 `props`나 상태가 변경되지 않는 한 리렌더링 되지 않고, 이전에 렌더링한 결과를 그대로 재사용
  
  $\rightarrow$ 복잡한 연산을 수행하지 않더라도 자주 렌더링 되는 컴포넌트 렌더링 횟수를 줄이는 데 유용

<br>

### `React.memo` 
- 컴포넌트를 메모이제이션할 때는 고차 컴포넌트인 `React.memo`를 사용
  - **고차 컴포넌트 (Higher Order Component)** : **컴포넌트를 인자로 받아 새로운 컴포넌트를 반환하는 함수**
- 반환된 컴포넌트는 `props`가 변경되지 않는 한 리렌더링 되지 않고, 이전 결과를 재사용

<br>

```ts
const MemoizedComponent = React.memo(MyComponent);
```

<br>

#### `React.memo` 적용

- `components/A.tsx`

```ts
import { memo } from "react";
import B from "./B";

export default memo(function A() {
    console.log('A render');
    return (
        <>
            <h1>A Component</h1>
            <B />
        </>
    )
});
```

<br>

#### `React.memo` 적용 주의사항

<br>

- **메모이제이션의 적절한 사용**
  - `React.memo`자체도 하나의 연산이기에 불필요하게 사용하면 오히려 성능 비용이 증가
  - 기본적으로 React는 상위 컴포넌트가 리렌더링되면 하위 컴포넌트도 함께 리렌더링
  
  $\rightarrow$ '`A` -> `B` -> `C`' 구조에서는 `A`만 `React.memo`만 적용하는 것이 효율적

<br>

- **메모이제이션의 무효화 조건**
  - **메모이제이션된 컴포넌트의 자체 상태가 변경된 경우**
  - **메모이제이션된 컴포넌트로 전달되는 `props` 값이 변경된 경우**
  
<br>

<hr>

<br>

## 03. 함수 메모이제이션

<br>

### 함수를 `props`로 전달하는 경우
- 자바스크립트에서 함수, 배열, 객체는 모두 **참조 자료형 (Reference Type)**
- **참조 자료형은 변수에 값 자체가 저장되는 것이 아니라, 값이 저장된 메모리 주소(참조값)가 저장**
  
  $\rightarrow$ **같은 로직의 함수라도 다시 정의하면 참조 값이 달라지게 됨**

- 예) `App.tsx`
  - **`App`컴포넌트가 렌더링될 때마다 `increment`함수는 새롭게 생성되며 참조 값이 달라짐**
  
  $\rightarrow$ **`React.memo`는 참조 값이 달라지면 `props` 값이 변경된 것으로 판단**

  $\rightarrow$ **메모이제이션 효과를 받지 못하고 리렌더링**

```ts
export default function App() {

    const increment = () => setCount((count) => count + 1);

    ...


}
```

<br>

### `useCallback` 훅
- **컴포넌트에 전달되는 `props` 중에는 실제로 화면 렌더링에 영향을 주지 않는 함수형 `props`가 존재**
  - 예) 이벤트 핸들러나 콜백 함수처럼 UI보다 동작에 관여하는 `props`
  
  $\rightarrow$ **매번 새로운 함수 객체가 리렌더링**

<br>

#### `useCallback` 훅 사용
- **함수의 참조 값을 고정**
    - **`fn` : 메모이제이션할 함수**
    - **`dependencies` : 의존성 배열**
      - 배열의 값이 변경되지 않는 한 함수는 새로 생성되지 않고 기존 것을 재사용
      - **의존성 배열을 빈 배열(`[]`)로 설정하면 컴포넌트가 처음 렌더링될 때 한 번만 함수가 생성되고, 리렌더링 때도 동일한 참조 값의 함수가 유지**

<br>

```ts
const cachedFn = useCallback(fn, dependencies);
```

<br>

- `App.tsx`

<br>

```ts
import { useState, useCallback } from "react";
import A from "./components/A";

export default function App() {
  console.log('App render');
  const [count, setCount] = useState(0);

  // useCallback 적용
  const increment = useCallback(() => setCount((count) => count + 1), []);
  
  return (
    <>
      <h1>App Count: {count}</h1>
      <A increment={increment} />
    </>
  )
}
```

<br>

#### `useCallback` 적용 주의사항
- **상태 변경 함수를 콜백 함수 형태로 사용**
  - **`useState`훅으로 정의한 상태를 변경할 때 이전 상태값을 기준으로 새 값을 계산해야 하는 경우에는, 상태 변경함수를 콜백 함수 형태로 사용하는 것이 안전**

<br>

- **의존성 배열에 상태 값을 포함하지 않기**

<br>

<hr>

<br>

## 04. 값 메모이제이션

<br>

### 연산 비용이 큰 작업의 성능 저하 문제 
- 컴포넌트 내부에 연산 비용이 큰 코드가 있으면, 리렌더링이 발생할 때마다 성능 저하가 누적되어 사용자 경험에 악 영향을 미칠 수 있음

<br>

### `useMemo` 훅
- `useCallback`은 함수 자체를 메모이제이션하는 반면, **`useMemo`는 함수의 반환 값을 메모이제이션**
    - **`calculateValue` : 메모이제이션할 값을 계산하는 함수** 
      - 값을 직접 전달하는 것이 아닌, 이 함수의 반환 값을 메모이제이션
    - **`dependencies` : 의존성 배열**
      - 배열에 포함된 값 중 하나라도 변경되면 `calculateValue` 함수가 다시 실행되어 새로운 값을 계산하고 저장
      - 배열의 값이 변경되지 않으면 기존에 저장된 값을 그대로 재사용

<br>

```ts
const cachedValue = useMemo(calculateValue, dependencies);
```

<br>

- `src/lib/utils.ts`

```ts
export const initialItems = new Array(29_999).fill(0).map((_, i) => {
    return {
        id: i,
        selected: i === 29_998
    };
});
```

<br>

- `App.tsx`

```ts

```